# Mathematical Foundations of Cryptography

## Modular Arithmetic

A number $x \bmod N$ is the equivalent of asking for the remainder of $x$ when divided by $N$. 

Two integers $a$ and $b$ are said to be **congruent** (or in the same equivalence class) modulo $N$ if they have the same remainder upon division by $N$. In such a case, we say that 

$$a \equiv b \pmod N$$

<br>

### Addition in Modular Arithmetic

1. If $a + b = c$, then $a \pmod N + b \pmod N \equiv c \pmod N$

2. If $a \equiv b \pmod N$, then $a + k \equiv b + k \pmod N$ for any integer $k$

3. If $a \equiv b \pmod N$ and $c \equiv d \pmod N$, then $a + c \equiv b + d \pmod N$

4. If $a \equiv b \pmod N$, then $-a \equiv -b \pmod N$

<br>

### Multiplication in Modular Arithmetic

1. If $a \cdot b = c$, then $a \pmod N \cdot b \pmod N \equiv c \pmod N$

2. If $a \equiv b \pmod N$, then $k \cdot a \equiv k \cdot b \pmod N$ for any integer $k$

3. If $a \equiv b \pmod N$ and $c \equiv d \pmod N$, then $a \cdot c \equiv b \cdot d \pmod N$

<br>

### Exponentiation in Modular Arithmetic

1. If $a \equiv b \pmod N$, then $a^k \equiv b^k \pmod N$ for any integer $k$

<br>

### Division in Modular Arithmetic

1. If $\gcd(k, N) = 1$ and $k \cdot a \equiv k \cdot b \pmod N$, then $a \equiv b \pmod N$

This property is true, because if $k \cdot (a−b)$ is a multiple of $N$ and $\gcd(k, N) = 1$, then $N$ must divide $a-b$, or equivalently $a \equiv b \pmod N$.

**Example**: Consider $4 \equiv 8 \pmod 4$. Note that we cannot simply divide both sides of the equation by $2$, since $2 \not \equiv 4 \pmod 4$.

<br>

### Multiplicative Inverse in Modular Arithmetic

If $a$ and $N$ are integers such that $\gcd(a, N) = 1$ (coprime or relatively prime), then there exists an integer $b$ such that $a \cdot b \equiv 1 \pmod N$. $b$ is called the multiplicative inverse of $a \bmod N$.

**Examples**:

* $2 \cdot 3 \equiv 1 \pmod{5}$
* $2 \cdot 6 \equiv 1 \pmod{11}$

In [7]:
# reference: https://en.wikipedia.org/wiki/Extended_Euclidean_algorithm#Pseudocode
def mod_inv(x, m):
    r0, r1 = x, m
    s0, s1 = 1, 0
    while r1 != 0:
        quotient = r0 // r1
        r0, r1 = r1, r0 - quotient * r1
        s0, s1 = s1, s0 - quotient * s1
    if r0 != 1:
        return None
    return s0 if s0 >= 0 else s0 + m

In [8]:
mod_inv(2, 11)

6

## The XOR Operation

The boolean XOR (exclusive OR) operation form an **abelian group** $(\,\{T, F\}^N, \,\oplus\,)$ over the set of **boolean vectors** of length $N$:

* **Closure**: when you XOR two boolean vectors $A$ and $B$, the result $C$ is also a boolean vector

* **Commutative**: $A \oplus B = B \oplus A$

* **Associative**: $A \oplus (B \oplus C) = (A \oplus B) \oplus C$

* The **identity element** for the XOR operation is $T^N$ with $A \oplus T^N = T^N \oplus A = A$

* **Inverse element**: each element is its own inverse, i.e. $A \oplus A = T^N$

### Isomorphism

Two groups are said to be **isomorphic** if there is a **one-to-one mapping** between the **elements** of the sets that **preserves the operation**.

<br>

The group $(\,\{T, F\}^N, \,\oplus\,)$ is isomorphic to the group $(\,\{0, 1\}^N, \, + \,)$ of **addition modulo 2** over the set of vectors whose **elements** are **integers mod 2**. 

&rarr; The isomorphism simply maps $T$ to $1$ and $F$ to $0$.

In [9]:
print((0 + 0) % 2, 0 ^ 0)
print((1 + 0) % 2, 1 ^ 0)
print((0 + 1) % 2, 0 ^ 1)
print((1 + 1) % 2, 1 ^ 1)

0 0
1 1
1 1
0 0


<br>

The group $(\,\{T, F\}^N, \,\oplus\,)$ is also isomorphic to the group $(\,\text{S}^N, \, \Delta \;)$ of **symmetric difference** $\Delta$ over the **power set** of $N$ elements (set of all possible subsets of $\text{S}$).

Symmetric difference is a **set** operation that gives the set of elements that are in either of the sets but **not in their intersection**:

$$
\begin{align}
A \, \Delta \, B &= (A / B) \cup (B / A) \qquad \text{or} \\[8pt]
A \, \Delta \, B &= (A \cup B) / (B \cap A)
\end{align}
$$

&rarr; The isomorphism maps $T$ to *included in the set* and $F$ to *excluded from the set* for each of the $N$ entries of the Boolean vector.



### Properties of the XOR Operation

#### Toggling

The combined value $A \oplus B$ *remembers* both states, and one state is the key to getting at the other:

$$
\begin{align}
A \oplus (A \oplus B) &= (A \oplus A) \oplus B \\[6pt]
&= 0 \oplus B \\[6pt]
&= B
\end{align}
$$

and

$$
\begin{align}
B \oplus (A \oplus B) &= B \oplus (B \oplus A) \\[6pt]
&= (B \oplus B) \oplus A \\[6pt]
&= 0 \oplus A \\[6pt]
&= A
\end{align}
$$

#### Parity

A powerful interpretation of XOR is in terms of parity, i.e. whether something is odd or even. 

For any $n$ bits, $A_1 \oplus A_2 \oplus \ldots \oplus A_n = 1$ if and only if the number of $1$s is **odd**.

<br>

An application of XOR’s parity property is **RAID** (Redundant Arrays of Inexpensive Disks) which is a way to recover from hard drive corruption. 

If we have $n$ hard drives, we can create an **additional** one $A^*$ which contains the XOR value of all the others:

$$
A^* = A_1 \oplus A_2 \oplus \ldots \oplus A_n
$$

This introduces redundancy: if a failure occurs on one drive, say $A_1$, we can **restore** it from **the others** since:

$$
\begin{align}
A_2 \oplus \ldots \oplus A_n \oplus A^* &= A_2 \oplus \ldots \oplus A_n \oplus (A_1 \oplus A_2 \oplus \ldots \oplus A_n) \\[6pt]
&= A_1 \oplus (A_2 \oplus A_2) \oplus \ldots \oplus (A_n \oplus A_n) \\[6pt]
&= A_1 \oplus 0 \oplus \ldots \oplus 0 \\[6pt]
&= A_1
\end{align}
$$

(this is the same reasoning used to explain toggling earlier, but applied to $n$ inputs rather than just $2$)

In the (highly unlikely) event that 2 drives fail simultaneously, the above would not be applicable so there would be no way to recover the data.

# Cryptography Principles

## Kerckhoffs's Principle

TODO

## Key-Space

The **key space** should be large enough to prevent **brute-force** and exhaustive-search attacks.

# Encryption Schemes

A encryption scheme $(\text{Gen}, \text{Enc}_k, \text{Dec}_k)$ is defined by a **message space** $\textbf{M}$ with the algorithms:

1. **Key generation** $\text{Gen}$ which generates a **key** $k$,

2. **Encryption** $\text{Enc}_k$ which takes key $k$ and **message** $m \in \textbf{M}$ as input and outputs a **ciphertext** $c$

$$c \gets \text{Enc}_k (m)$$

3. **Decryption** $\text{Dec}_k$ which takes key $k$ and ciphertext $c$ as input and outputs message $m$ (or "error")

$$m := \text{Dec}_k (c)$$

and

$$\text{Dec}_k(\text{Enc}_k(m)) = m$$

<br>

Here 
* the message space $\textbf{M}$ is the set of all possible messages,

* the key space $\textbf{K}$ is the set of all possible keys,

* the ciphertext space $\textbf{C}$ is the set of all possible ciphertexts,

* the **left arrow** $\gets$ notation denotes assignment to the output of an algorithm that might be **randomized**. Meaning that the output of the algorithm may be different, even when run twice on the same set of inputs,

* the **colon equals** $:=$ denotes an assignment to the output of a **deterministic** algorithm and

* a single **equal sign** $=$ denotes mathematical equality in contrast to an assignment.

## On Probability

* a **random valriable** is a variable that takes on (discrete) values with certain **probabilities**.

* a **probability distribution** of a random variable specifies the probabilities with which the variable taes on each possible value

* an **event** $\text{E}$ is a particular occurence in some experiment with the probability $\text{P}[\,\text{E}\,]$

* the probability that one event occurs, *assuming* some other event occured is called **conditional probability** ( $\cap$ corresponds to "and")

$$\text{P}[\,\text{A}\, | \,\text{B}\,] = \frac{\text{P}[\,\text{A}\, \cap \text{B}\,]}{\text{P}[\,\text{B}\,]}$$

* two random variables $\text{A}$ and $\text{B}$ are **independent** if for all $a$ and $b$

$$\text{P}[\,\text{A} = a\, | \,\text{B} = b\,] = \text{P}[\,\text{A} = a\,]$$

* **Law of total probability**: given that $\text{E}_1, \ldots, \text{E}_n$ are *partitions* of all possibilities, then for any $\text{A}$

$$
\begin{align}
\text{P}[\,\text{A}\,] &= \sum_i \text{P}[\,\text{A} \cap \text{E}_i\,] \qquad \text{or alternatively} \\[6pt]
&= \sum_i \text{P}[\,\text{A}\, | \,\text{E}_i\,] \cdot \text{P}[\,\text{E}_i\,]
\end{align}
$$

* **Bayes Theorem**

$$
\text{P}[\,\text{A}\, | \,\text{B}\,] = \frac{\text{P}[\,\text{B}\, | \,\text{A}\,] \cdot \text{P}[\,\text{A}\,]}{\text{P}[\,\text{B}\,]}
$$


## Probability Distributions of Encryption Schemes

Let $\text{M}$ be a **random variable** denoting the value of a message from the **message space** $\textbf{M}$ (set of all possible messages).

This reflects the likelyhood of different messages sent by the parties, given the **attackers prior knowledge**, e.g.

$$
\begin{align}
&\text{P}[\, \text{M} = \text{attack today}\,] = 0.7 \\
&\text{P}[\, \text{M} = \text{do not attack}\,] = 0.3
\end{align}
$$

<br>

Let $\text{K}$ be a **random variable** denoting the key from the **key space** $\textbf{K}$ (set of all possible keys).

Given a **encryption scheme** $(\text{Gen}, \text{Enc}_k, \text{Dec}_k)$, then $\text{Gen}$ defines a **probability distribution** for $\text{K}$:

$$
\text{P}[\,\text{K} = k\,] = \text{P}[\,\text{Gen outputs key}\; k\,]
$$

**Example**: consider the shift cipher where the key space is $\textbf{K} = (0, \ldots, 25)$, then $\text{P}[\, \text{K} = 15\,] = 1/26$ 

Here we make the reasonable **assumption**, that $\text{M}$ and $\text{K}$ are **independent** variables, i.e. the message that a party sends does not depend on the key used to encrypt the message.

<br>

Given an encryption scheme $(\text{Gen}, \text{Enc}_k, \text{Dec}_k)$, then a message $m$ according the given distribution $\text{M}$ defines a **distribution** of the **ciphertext**.

Let $\text{C}$ be the **random variable** denoting the value of the ciphertext from the **ciphertext space** $\textbf{C}$ (set of all possible ciphertexts).

<br>

### Example 1

Given a **shift cipher**, then for all $k \in \{0, \ldots, 25\}$ we have $\text{P}[\, \text{K} = k\,] = 1/26$.

Say we have the distribution

$$
\begin{align}
&\text{P}[\,\text{M} = \text{a}\,] = 0.7 \\
&\text{P}[\, \text{M} = \text{z}\,] = 0.3
\end{align}
$$

then what is $\text{P}[\, \text{C} = \text{b}\,]$?

The ciphertext $\text{C}$ can only be $\text{b}$, if $\text{M} = \text{a}$ and $\text{K} = 1$ or if $\text{M} = \text{z}$ and $\text{K} = 2$, hence

$$
\begin{align}
\text{P}[\,\text{C} = \text{b}\,] &= \text{P}[\,\text{M} = \text{a}\,] \cdot \text{P}[\,\text{K} = 1\,] + \text{P}[\,\text{M} = \text{z}\,] \cdot \text{P}[\,\text{K} = 2\,] \\[7pt]
&= 0.7 \cdot 1/26 + 0.3 \cdot 1/26 \\[7pt]
&= 1/26 \\
\end{align}
$$

### Example 2

Consider a **shift cipher** and the message distribution 

$$
\begin{align}
&\text{P}[\,\text{M} = \text{one}\,] = 0.5 \\
&\text{P}[\, \text{M} = \text{ten}\,] = 0.5
\end{align}
$$

What is $\text{P}[\, \text{C} = \text{rqh}\,]$?

Due to the **law of total probability** we calculate

$$
\begin{align}
\text{P}[\,\text{C} = \text{rqh}\,] &= \text{P}[\,\text{C} = \text{rqh}\, | \, \text{M} = \text{one}\,] \cdot \text{P}[\, \text{M} = \text{one}\,] + \text{P}[\,\text{C} = \text{rqh}\, | \, \text{M} = \text{ten}\,] \cdot \text{P}[\, \text{M} = \text{ten}\,] \\[7pt]
&= 1/26 \cdot 0.5 + 0 \cdot 0.5 \\[7pt]
&= 1/52 \\
\end{align}
$$

# Core Principles of Modern Cryptography

1. **Formal Definitions**: precise mathematical model and definition of what security means

2. **Assumptions**: clearly stated and unambigious

3. **Proofs of Security**: move away from design-break-patch

# Perfect Secrecy

## Informal Definition of Perfect Secrecy

> "Regardless of any **prior** information the attacker has about the plaintext, the ciphertext should leak **no additional** information about the plaintext"

<br>

## Formal Definition of Perfect Secrecy

A message scheme $(Gen, Enc_k, Dec_k)$ with message space $\textbf{M}$ and ciphertext space $\textbf{C}$ is **perfectly secret** if, for every probability distribtion over $\textbf{M}$, every $m \in \textbf{M}$ and every $c \in \textbf{C}$ with $\text{P}[\, \text{C} = c\,] > 0$ it holds that the *a posteriori* (a.k.a. *conditional*) probability that the message $m$ conditioned on the observed ciphertext being equal to $c$ is exactly equal to the *apriori* probability that the message is equal to $m$ (where this *apriori* probability is exactly the given probability distribution, that we started with):

$$
\text{P}[\, \text{M} = m\, | \, \text{C} = c \,] = \text{P}[\, \text{M} = m \,]
$$

### Example 3

Consider a **shift cipher** and the message distribution

$$
\begin{align}
&\text{P}[\,\text{M} = \text{one}\,] = 0.5 \\
&\text{P}[\, \text{M} = \text{ten}\,] = 0.5
\end{align}
$$

Now, take as one particular message $m = \text{ten}$ and as one particular ciphertext $c = \text{rqh}$.

What is the **probability** that the message is equal to $\text{ten}$ conditioned on the fact that we **observed** a ciphertext $\text{rqh}$?

Because there is no key that match the message $m$ to the ciphertext $c$, that mean that if we observe to ciphertext $c$ it is not possible that the message is equal to $m$, hence it is not equal to the *prior probability* that the message is $\text{ten}$:

$$
\begin{align}
\text{P}[\, \text{M} = \text{ten}\, | \, \text{C} = \text{rqh} \,] &= 0 \\[7pt]
&\neq \text{P}[\, \text{M} = \text{ten} \,]
\end{align}
$$

This shows that the **shift cipher** does **not meet the definition** of **perfect secrecy**.

### Example 4

Consider a **shift cipher** and the message distribution

$$
\begin{align}
&\text{P}[\,\text{M} = \text{hi}\,] = 0.3 \\
&\text{P}[\,\text{M} = \text{no}\,] = 0.2 \\
&\text{P}[\, \text{M} = \text{in}\,] = 0.5
\end{align}
$$

What is the **probability** $\text{P}[\, \text{M} = \text{hi}\, | \, \text{C} = \text{xy} \,]$ that the message is equal to $\text{hi}$ conditioned on the fact that we **observed** a ciphertext $\text{xy}$?

With

$$
\begin{align}
\text{P}[\, \text{C} = \text{xy}\, | \, \text{M} = \text{hi} \,] = 1/26
\end{align}
$$

and due to the law of total probability

$$
\begin{align}
\text{P}[\,\text{C} = \text{xy} \,] &= \text{P}[\, \text{C} = \text{xy}\, | \, \text{M} = \text{hi} \,] \cdot 0.3 + \text{C} = \text{xy}\, | \, \text{M} = \text{no} \,] \cdot 0.2 + \text{C} = \text{xy}\, | \, \text{M} = \text{in} \,] \cdot 0.5 \\[7pt]
&= 1/26 \cdot 0.3 + 1/26 \cdot 0.2 + 0 \cdot 0.5 \\[7pt]
&= 1/52
\end{align}
$$

we can calculate using the Bayes theorem

$$
\begin{align}
\text{P}[\, \text{M} = \text{hi}\, | \, \text{C} = \text{xy} \,] &= \frac{\text{P}[\, \text{C} = \text{xy}\, | \, \text{M} = \text{hi} \,] \cdot \text{P}[\,\text{M} = \text{hi} \,]}{\text{P}[\,\text{C} = \text{xy} \,]} \\[7pt]
&= \frac{1/26 \cdot 0.3}{1/52} \\[7pt]
&= 0.6 \\[7pt]
&\neq \text{P}[\, \text{M} = \text{hi} \,]
\end{align}
$$

This again shows that the **shift cipher** is **not perfectly secret**.

### One-Time Pad

The one-time pad **encryption scheme** is the following:

* Let $\textbf{M} = \{0, 1\}^n$ (the set of all binary strings of length $n$, i.e. an $n$-bit string)

* $\text{Gen}$: choose an uniform **key** $k \in \{0, 1\}^n$. Each possible $n$-bit string is chosen with probability $\text{P}[K] = 2^{-n}$. There are $2^n$ different strings of length $n$.

* $\text{Enc}_k(m) = k \oplus m$ (bit-wise XOR)

* $\text{Dec}_k(c) = k \oplus c$

This scheme is **correct**, as

$$
\begin{align}
\text{Dec}_k(\text{Enc}_k(m)) &= k \oplus (k \oplus m) \\[6pt]
&= (k \oplus k) \oplus m \\[6pt]
&= 0 \oplus m \\[6pt]
&= m
\end{align}
$$

#### Perfect Secrecy of the One-Time Pad

The encryption scheme that achieves **perfect secrecy** is the **one-time pad**. It was proven perfectly secret by **Shannon** in 1949.

**Prove**:

Let's agree on some **arbitrary distribution** over the message space $\textbf{M} \in \{0, 1\}^N$, and agree on some **arbitrary message** $m$ and **arbitrary cyphertext** $c$.

Then using the **law of total probability** we can calculate the probability of the ciphertext $c$ by summation over all possible messages $m'$:

$$
\begin{align}
\text{P}[\, \text{C} = c \,] &= \sum_{m'} \text{P}[\, \text{C} = c \, | \, \text{M} = m' \,] \cdot \text{P}[\, \text{M} = m' \,] \\[6pt]
&= \sum_{m'} \text{P}[\, \text{K} = m' \oplus c \,] \cdot \text{P}[\, \text{M} = m' \,] \\[6pt]
&= \sum_{m'} 2^{-n} \cdot \text{P}[\, \text{M} = m' \,] \\[6pt]
&= 2^{-n}
\end{align}
$$

We then calculate using Bayes Law

$$
\begin{align}
\text{P}[\, \text{M} = m \, | \, \text{C} = c \,] &= \frac{\text{P}[\, \text{C} = c \, | \, \text{M} = m \,] \cdot \text{P}[\, \text{M} = m\,]}{\text{P}[\text{C} = c \,]} \\[6pt]
&= \frac{\text{P}[\, \text{K} = m \oplus c \,] \cdot \text{P}[\, \text{M} = m\,]}{2^{-n}} \\[6pt]
&= \frac{2^{-n} \cdot \text{P}[\, \text{M} = m\,]}{2^{-n}} \\[6pt]
&= \text{P}[\, \text{M} = m\,]
\end{align}
$$